# 复杂纠缠态制备

前面介绍了单模线路结合时分复用方法制备简单的纠缠态，包括$EPR$态和$GHZ$态，这里介绍多模线路结合时分复用方法制备复杂的纠缠态。

## 1. 案例一

第一种纠缠态的制备线路如下图所示， 由东京大学组提出[1]， 通过两模线路结合延时线圈，实现了四个测量结果的纠缠，即实现了相邻两个时刻的两组测量结果纠缠。
<div style="margin-right: 15px; border-radius: 10px; background-color: rgb(255， 255， 255); text-align: center;">
    <img src="./fig/1.jpg" width="30%"/>
    <p style="padding: 10px; font-size: small; text-align: center; line-height: 0%;">
        <b>
    </p>
</div>
下面给出这种纠缠态制备线路的理论分析以及使用Deepquantum代码复现。 

### 理论分析
这里我们在薛定谔表象中研究量子态如何经过一系列光量子门后演化到纠缠态。
1. 初态

$k$ 时刻的初态可以表示为
$$
\psi_k = |x=0\rangle^A_k|p=0\rangle^B_k = \int_{-\infty}^{+\infty}|x=0\rangle^A_k|x=a_k\rangle^B_k da_k
$$
那么考虑多个时刻的量子态初态为
$$
\Psi_0 = \prod_k \int_{-\infty}^{+\infty}|x=0\rangle^A_k|x=a_k\rangle^B_k da_k
$$

2. 第一个分束器

经过第一个$1:1$ 分束器，相同时刻实现空间上的纠缠

$$
\Psi_1 = \hat{BS} \Psi_0 = \prod_k \int_{-\infty}^{+\infty}|x=\frac{a_k}{\sqrt{2}}\rangle^A_k |x=\frac{a_k}{\sqrt{2}}\rangle^B_k da_k
$$
具体可以写成
$$
\Psi_1 =\int_{-\infty}^{+\infty}\int_{-\infty}^{+\infty}\int_{...}|x=\frac{a_1}{\sqrt{2}}\rangle^A_1 |x=\frac{a_1}{\sqrt{2}}\rangle^B_1 
|x=\frac{a_2}{\sqrt{2}}\rangle^A_2 |x=\frac{a_2}{\sqrt{2}}\rangle^B_2 da_1da_2......
$$
3. 延时线圈

作用在模B上的延时线圈仅仅起到延时作用， 没有分束器起作用
$$
\Psi_2 =\int_{-\infty}^{+\infty}\int_{-\infty}^{+\infty}\int_{...}|x=\frac{a_1}{\sqrt{2}}\rangle^A_1 |x=\frac{a_1}{\sqrt{2}}\rangle^B_2 
|x=\frac{a_2}{\sqrt{2}}\rangle^A_2 |x=\frac{a_2}{\sqrt{2}}\rangle^B_3 da_1da_2......
$$

4. 第二个分束器

经过第二个$1:1$ 分束器，相同时刻实现空间上的纠缠
$$
\Psi_2 =\int_{-\infty}^{+\infty}\int_{-\infty}^{+\infty}\int_{...}|x=\frac{a_1}{\sqrt{2}}\rangle^A_1 |x=\frac{1}{2}(a_2-a_1)\rangle^B_2 
|x=\frac{1}{2}(a_2+a_1)\rangle^A_2 |x=\frac{1}{2}(a_3-a_2)\rangle^B_3
|x=\frac{1}{2}(a_3+a_2)\rangle^A_3 da_1da_2da_3......
$$

考虑相邻两次的测量结果，
$$
\frac{1}{2}(a_k-a_{k-1}) + \frac{1}{2}(a_k+a_{k-1}) + \frac{1}{2}(a_{k+1}-a_{k}) - \frac{1}{2}(a_{k+1}+a_{k}) = 0
$$
可以看到相邻的两次测量结果关联起来了，形成了四个量子态组成的纠缠态。

### 代码演示

In [1]:
import deepquantum as dq
import numpy as np
import torch

In [2]:
r = 6
nmode = 2
circ = dq.QumodeCircuit(nmode=nmode, init_state='vac', cutoff=3, backend='gaussian', basis=True)
circ.s([0], r=r)
circ.s([1], r=r)
circ.r([0], np.pi/2)
circ.bs([0,1], [np.pi/4, 0])
circ.delay([1], [1, [torch.tensor(np.pi/2)],[torch.tensor(np.pi)]])# r = pi 做相位修正, 等价于Cross 状态
circ.bs([0,1], [np.pi/4, 0])
circ.homodyne_x([0])
circ.homodyne_x([1])
test = circ()
circ.draw()

完成线路前向运行之后可以查看等效的线路

In [8]:
circ.unfold_circuit.draw()

In [9]:
shots = 20
samples = circ.measure_homodyne(shots=shots)
samples

tensor([[ 508.2537,   47.0051],
        [-570.0271,  -14.7648],
        [ 127.1648, -457.6379],
        [ -56.0129, -386.4845],
        [ 343.7956,  -98.7048],
        [ -84.2959,  160.7964],
        [-173.3916,  -96.9020],
        [  14.2454, -256.0489],
        [ -36.6883, -278.4984],
        [ -73.9323, -389.1119],
        [ 531.6588,   68.6159],
        [-559.2243,   41.0485],
        [ 324.2510, -193.9199],
        [  92.3418,  222.6711],
        [-118.2236,  196.7890],
        [-181.9926, -103.4279],
        [ 514.0080,  228.5854],
        [-243.4348,  499.1600],
        [-245.6557,   10.0707],
        [-177.9113, -413.4923]], dtype=torch.float64)

计算相邻四个结果的误差

In [10]:
size = int(shots/2)
samples_reshape = samples.reshape(size,2,2)
x_delta = [ ]
for i in range(size):
    temp = samples_reshape[i].sum() - 2*samples_reshape[i][1,1]
    x_delta.append(temp)
x_delta2 = torch.stack(x_delta)
print('variance:', x_delta2.std()**2)
print('mean:', x_delta2.mean())

variance: tensor(8.0398e-06, dtype=torch.float64)
mean: tensor(-0.0014, dtype=torch.float64)


# 附录

[1] Yokoyama S, Ukai R, Armstrong S C, et al. Ultra-large-scale continuous-variable cluster states multiplexed in the time domain[J]. Nature Photonics, 2013, 7(12): 982-986.

[3] Asavanant W. Time-Domain Multiplexed 2-Dimensional Cluster State: Universal Quantum Computing Platform[J]. arxiv. org, Cornell University Library, 201.